<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>
<a href="https://github.com/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
Use this notebook to get alignment information from ImageJ image registration. Print stats for individual barcodes and calculate mapping from u,v pixels to x,y,z physical space.

In [2]:
# @title Notebook Inputs { display-mode: "form" }
# @markdown How to use this notebook: [See Instructions](https://docs.google.com/document/d/1G2AME1q6XQhxQ1A2FhkfpktaSFpNXSNQB6mEWwM0YM0/edit?usp=sharing)
import numpy as np
from google.colab import drive
from google.colab import files
drive.mount('/content/drive/')


template_size_px = 401


# @markdown Input Paths:
# @markdown Leave either path blank to load a file from local file system.
alignment_xml_path = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/align8.xml" # @param {type:"string"}
fluorescent_patch_number = 8 # @param {type:"integer"}
# @markdown For the alignment of multiple templates to one fluorescent image, specify the patch numbers of each template in the TrakEM stack.
# template_patch_1 = 11 # @param {type:"integer"}
# template_patch_2 = 14 # @param {type:"integer"}
# template_patch_3 = 17 # @param {type:"integer"}

# @markdown Enter template patch IDs in order as a comma-separated list. Example: [11, 14, 17]
template_patch_numbers = [11, 14, 17, 20, 23, 26, 29, 32] # @param

# @markdown Z-depth of each template, in um, as a comma-separated list. Example: [50, 52, 54]
template_centers_z_um = [32, 34, 32, 32, 34, 34, 34, 36] # @param
template_centers_z = [x/1000 for x in template_centers_z_um] # to mm

# @markdown Real (x,y) locations in mm of photobleach barcode centers. Enter in format: [0,0], [1,0], [2,0], [0,1].
template_centers_xy_mm = [0,0], [1,0], [2,0], [0,1], [1,1], [2,1], [1,2], [2,2] # @param

template_centers_xy = np.array(template_centers_xy_mm)
assert template_centers_xy.shape[1] == 2, "Points in template_centers_xy should be in format [x,y]"

assert len(template_patch_numbers) == len(template_centers_z), "Number of elements in template patch list and template z list must match"

if not alignment_xml_path:
  print("Upload saved TrakEM project:")
  uploaded = files.upload()
  alignment_xml_path = list(uploaded.keys())[0]
  alignment_xml_path = os.path.join(os.getcwd(), alignment_xml_path)


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# @title Environment Setup
!git clone https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

from plane.fit_template import FitTemplate
from plane.fit_plane import FitPlane
import matplotlib.pyplot as plt
import os
from google.colab import files
import math
import pandas as pd
import numpy as np

%cd ..

Cloning into 'coregister-xy'...
remote: Enumerating objects: 802, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 802 (delta 194), reused 208 (delta 163), pack-reused 531 (from 1)
Receiving objects: 100% (802/802), 1.24 MiB | 10.84 MiB/s, done.
Resolving deltas: 100% (502/502), done.
/content/coregister-xy
/content


In [4]:
# @title Print Template Stats

# Setup code
fts = []
templates = [int(x) for x in template_patch_numbers]

for i in range(0, len(templates)):
    ft = FitTemplate.from_imagej_xml(alignment_xml_path, fluorescent_patch_number, templates[i], None, True)
    fts.append(ft)

uv_px = []
for i, ft in enumerate(fts):
    uv_px.append((ft.tx + template_size_px/2, ft.ty + template_size_px/2))

zs = np.array([float(x) for x in template_centers_z])
template_centers_xyz = np.column_stack((template_centers_xy, zs))
template_centers_uv = np.array(uv_px)

fp = FitPlane.from_template_centers(template_centers_uv, template_centers_xyz, print_inputs = False)


"""
Print stats for each FitPlane as a table: shrinkage, rotation, shear, and mean/stdev for each
"""
num_templates = len(fts)
projects_data = {
"Template ID": [i for i in range(1, num_templates+1)],
"Center (x, pix)": [project.tx + template_size_px/2 for project in fts],
"Center (y, pix)": [project.ty + template_size_px/2 for project in fts],
"Rotation (deg)": [project.theta_deg for project in fts],
"Scaling (um per pixel)": [project.scale for project in fts],
"Shear magnitude": [project.shear_magnitude for project in fts],
"Shear unit vector (x)": [project.shear_vector[0] for project in fts],
"Shear unit vector (y)": [project.shear_vector[1] for project in fts]
}

columns_to_summarize = ["Rotation (deg)", "Scaling (um per pixel)", "Shear magnitude", "Shear unit vector (x)", "Shear unit vector (y)"]

# Create DataFrame
df = pd.DataFrame(projects_data)

# Compute mean and standard deviation for selected columns only
mean_row = df[columns_to_summarize].mean()
std_row = df[columns_to_summarize].std()

# Append mean and std as new rows for selected columns only
summary_df = df.copy()
summary_df.loc['Mean', columns_to_summarize] = mean_row
summary_df.loc['StDev', columns_to_summarize] = std_row
summary_df = summary_df.round(2)
summary_df = summary_df.replace(np.nan, '', regex=True)

summary_df


,Template ID,"Center (x, pix)","Center (y, pix)",Rotation (deg),Scaling (um per pixel),Shear magnitude,Shear unit vector (x),Shear unit vector (y)
0,1.0,644.31,1062.96,11.78,1.66,0.04,0.80,0.60
1,2.0,1454.47,1166.28,10.43,1.77,0.03,0.73,0.68
2,3.0,2425.81,1332.18,11.99,1.83,0.14,0.99,0.11
3,4.0,532.06,1851.14,13.16,1.79,0.07,1.00,0.02
4,5.0,1360.98,2003.33,9.54,1.88,0.07,0.96,0.28
5,6.0,2300.31,2129.16,11.40,1.93,0.04,0.95,0.32
6,7.0,1190.89,2833.23,13.24,1.86,0.09,1.00,0.08
7,8.0,2125.52,3012.87,11.73,1.97,0.06,0.95,-0.30
Mean,,,,11.66,1.84,0.07,0.92,0.22
StDev,,,,1.25,0.10,0.04,0.10,0.32


In [5]:
# @title Plane Mapping Stats
# Print stats
print('Fit Statistics:')
print('---------------')
print("Pixel Size: |u| = %.3fum, |v| = %.3fum" %
  (fp.u_norm_mm()*1000, fp.v_norm_mm()*1000 ) )
print(f"Signed Distance from Origin: {fp.distance_from_origin_mm() * 1000:.3f}um")
print("X-Y Rotation: %.2f degrees" % fp.xy_rotation_deg())
print(f"Tilt: {90-fp.tilt_deg():.3f} degrees")

# Serialize to JSON
print("")
print('Data to Keep:')
print('---------------')
print("fp = FitPlane.from_json('" + fp.to_json() + "')")
print()
print()

print("To copy and paste into reconstruct_3d.ipynb:")
print('---------------')
print(f"plane_inputs.append({{\n\"uv_pix\": {template_centers_uv.tolist()},\n\"xyz_mm\": {template_centers_xyz.tolist()},\n\"image_file_path\": \"INSERT_IMAGE_PATH_HERE.png\"\n}})")

# Write data to JSON for easy loading

outfile = "save_mapping.json" # Change filepath as needed

json_data = fp.to_json()
with open(outfile, 'w') as f:
    f.write(json_data)

Fit Statistics:
---------------
Pixel Size: |u| = 1.107um, |v| = 1.186um
Signed Distance from Origin: 31.890um
X-Y Rotation: 9.84 degrees
Tilt: 0.062 degrees

Data to Keep:
---------------
fp = FitPlane.from_json('{"u": [0.0010910388967059505, -0.00018930164426477956, 5.389558699567997e-07], "v": [0.0001907877197614561, 0.0011707269179640969, 1.2787894742598143e-06], "h": [-0.8832992011346803, -1.0925888019513494, 0.030228996490115137]}')


To copy and paste into reconstruct_3d.ipynb:
---------------
plane_inputs.append({
"uv_pix": [[644.3145544392017, 1062.957667902862], [1454.4749105294493, 1166.278689622642], [2425.8069707349687, 1332.1824696121407], [532.0592340822607, 1851.1354815535324], [1360.9773198767193, 2003.333188393078], [2300.3140697037047, 2129.1634252115546], [1190.8930267023206, 2833.226498860065], [2125.5165130943883, 3012.872838578435]],
"xyz_mm": [[0.0, 0.0, 0.032], [1.0, 0.0, 0.034], [2.0, 0.0, 0.032], [0.0, 1.0, 0.032], [1.0, 1.0, 0.034], [2.0, 1.0, 0.034], [1.0, 

In [6]:
uv_to_xyz = np.array([fp.get_xyz_from_uv(x) for x in template_centers_uv])

in_plane_err, out_plane_err = fp.get_template_center_positions_distance_metrics(template_centers_uv, template_centers_xyz)

print(f"Average in-plane mapping error: {in_plane_err*1000:.3f} um")
print(f"Average out-of-plane mapping error: {out_plane_err*1000:.3f} um")

Average in-plane mapping error: 37.696 um
Average out-of-plane mapping error: 0.000 um


In [7]:
# @title Visualize Plane with Points
import plotly.graph_objects as go

# plane coeffs
normal_vector = np.cross(fp.u, fp.v)
a,b,c = normal_vector
d = -1 * (a*fp.h[0] + b*fp.h[1] + c*fp.h[2])

# Generate grid for the plane
maxlims = np.max(template_centers_xy, axis=0)
x = np.arange(-0.1, maxlims[0]*1.2, 0.1)
y = np.arange(-0.1, maxlims[1]*1.2, 0.1)
X, Y = np.meshgrid(x, y)
Z = (-d - a * X - b * Y) / c  # Solve for Z

# Create the plane surface
plane = go.Surface(x=X, y=Y, z=Z, colorscale='Viridis', opacity=0.7, name='Plane',showscale=False)

# Create the points
uv = go.Scatter3d(
    x=uv_to_xyz[:, 0],
    y=uv_to_xyz[:, 1],
    z=uv_to_xyz[:, 2],
    mode='markers',
    marker=dict(size=4, color='blue'),
    name='UV to XYZ'
)
xyz = go.Scatter3d(
    x=template_centers_xyz[:, 0],
    y=template_centers_xyz[:, 1],
    z=template_centers_xyz[:, 2],
    mode='markers',
    marker=dict(size=4, color='red'),
    name='True XYZ'
)
z_min = 0.8 * np.min(zs)
z_max = 1.2 * np.max(zs)

fig = go.Figure()
fig.add_trace(plane)
fig.add_trace(uv)
fig.add_trace(xyz)
fig.update_layout(
    scene=dict(
        xaxis_title='X (mm)',
        yaxis_title='Y (mm)',
        zaxis_title='Z depth (mm)',
        zaxis=dict(range=[z_max, z_min])
    )
)
fig.show()